In [8]:
LONG_TEXT = """Text literals and metacharacters make up this string. The compile function is used to create the pattern."""

In [9]:
from icecream import ic

In [10]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
from util import StringSpans

In [11]:
#@title random_bit_stream
import random

def random_bit_stream(length=None):
    """Return a random string of zeros and ones of the given length (default: random integer between 0 and 100)."""
    if length is None:
        length = random.randint(0, 100)
    return ''.join(str(random.randint(0, 1)) for _ in range(length))
def int_to_binary_string(n: int, length: int):
    binary_str = bin(n)[2:]  # convert to binary string, remove '0b' prefix
    padded_str = binary_str.rjust(length, '0')  # pad with zeros to length
    return padded_str

In [12]:
#@title https://github.com/farkmarnum/emojify
import json
import random
from math import log2,floor,ceil 
import itertools
import re
from typing import Dict, Generator, List, Tuple

with open('./emoji-data.json', 'r') as f:
    emoji_data: Dict[str,Dict[str,List[str]]] = json.load(f)

In [13]:
#@title Constants
regex = re.compile(r'[a-z0-9]+')
ALL_EMOJIS = set()
for k,v in emoji_data.items():
  if regex.match(k) is None:
    ALL_EMOJIS.add(k)
    # print('k',k)
  if isinstance(v,str) and regex.match(v) is None:
    ALL_EMOJIS.add(v)
    # print('v',v)
  else:
    for kk,vv in v.items():
      if regex.match(kk) is None:
        ALL_EMOJIS.add(kk)
        # print('kk',kk)
      if isinstance(vv,str) and regex.match(vv) is None:
        ALL_EMOJIS.add(v)
        # print('vv',vv)
EMOJIER_COMMON_WORDS = {
    'a',
    'an',
    'as',
    'is',
    'if',
    'of',
    'the',
    'it',
    'its',
    'or',
    'are',
    'this',
    'with',
    'so',
    'to',
    'at',
    'was',
    'and',
  }

In [14]:
#@title encode decode
class Emojier:
  @staticmethod
  def gaussian_order(lst):
    length = len(lst)
    max_odd_ind = length - 1 if length % 2 == 0 else length - 2
    max_even_ind = length - 1 if length % 2 != 0 else length - 2
    dist = itertools.chain(range(max_odd_ind,0,-2),range(0,max_even_ind + 1 , 2))
    return [lst[i] for i in dist]

  @staticmethod
  def encode(
        input_str: str,
        bytes_str: str,
        verbose=False,
        mask=True,
        maskStep: int =6,
        topX=False,
        X: float=0.15
    ) -> Tuple[str,str]:
    
    if verbose:
      print('encode:')
    input_str_spans = StringSpans(input_str)
    word_span_n_words = zip(input_str_spans.words, input_str_spans.get_words())
    result = input_str
    acc_offset = 0
    
    word_span_n_words_options: List[Tuple[int,str,List[str]]] = []
    for (_,we), word_raw in word_span_n_words:
      word = word_raw.lower()
      is_too_common = word in EMOJIER_COMMON_WORDS

      emoji_options = \
        Emojier.gaussian_order( ['']+
          [x[0] for x in
            sorted(
              emoji_data.get(word, {}).items(),
              key=lambda x:x[1],
              reverse=True
            )
          ]
        )
      if not is_too_common and len(emoji_options)>=2:
        word_span_n_words_options.append((we,word_raw, emoji_options))
    
    if mask:
      word_span_n_words_options = word_span_n_words_options[::maskStep]
    if topX:
      word_span_n_words_options.sort(key=lambda tup : len(tup[2]),reverse=True)
      taken_elements = ceil(len(word_span_n_words_options) * X) 
      word_span_n_words_options = word_span_n_words_options[:taken_elements]
      
    for we, word_raw, emoji_options in word_span_n_words_options:
      word = word_raw.lower()

      if verbose:
        print(f"word: {word} \nlen: {len(emoji_options)} \temoji_options[:10]: {emoji_options[:10]}")

      bits = floor(log2(len(emoji_options)))
      taken_bits = bytes_str[:bits]
      ind = int(taken_bits, 2)
      bytes_str = bytes_str[bits:]
      emojis = emoji_options[ind]
      if len(emojis) > 0:
        we = we + acc_offset
        acc_offset += len(emojis) + 1
        if verbose:
          print(f'>>>encoding {taken_bits} = {ind} as {emojis}\nwe={we}\tacc_offset={acc_offset}')
          print(f'result[:we]="{result[:we]}" result[we:]="{result[we:]}"')  
        result = f'{result[:we]} {emojis}{result[we:]}'  

    return result, bytes_str

  @staticmethod
  def eat_back(s:str) -> Generator[str,None,None]:
    for i in range(len(s),-1,-1):
      yield s[0:i]
  @staticmethod
  def decode(
            input_str: str,
            verbose=False,
            mask: bool =True,
            maskStep: int =6,
            topX: bool =False,
            X: float=0.15
      ) -> Tuple[str,str]:
    
    if verbose:
      print('decoding!')
    wordish = re.compile(r'^[a-z]*$')
    input_str_ss = StringSpans(input_str)
    words = [input_str[s:e] for s,e in input_str_ss.non_spaces]
    result = input_str
    bytes_str = ''
    
    emoticons_used = []
    word_span_n_words_options: List[Tuple[int,str,List[str]]] = []
    for i, word_raw in enumerate(words[:-1]):
      word = word_raw.lower()
      
      if wordish.match(word) is None:
        continue 

      is_too_common = word in EMOJIER_COMMON_WORDS

      emoji_options = \
        Emojier.gaussian_order( ['']+
          [x[0] for x in
            sorted(
              emoji_data.get(word, {}).items(),
              key=lambda x:x[1],
              reverse=True
            )
          ]
        )
      if not is_too_common and len(emoji_options) >= 2:
        word_span_n_words_options.append((i,word_raw,emoji_options))

    if mask:
      word_span_n_words_options = word_span_n_words_options[::maskStep]
    if topX:
      word_span_n_words_options.sort(key=lambda tup : len(tup[2]),reverse=True)
      taken_elements = ceil(len(word_span_n_words_options) * X) 
      word_span_n_words_options = word_span_n_words_options[:taken_elements]
        
    for i, word_raw, emoji_options in word_span_n_words_options:
      word = word_raw.lower()

      if verbose:
        print(f"word: {word} \nlen: {len(emoji_options)} \temoji_options[:10]: {emoji_options[:10]}")

      bits = floor(log2(len(emoji_options)))
      index = 0
      for w in Emojier.eat_back(words[i+1]):
        if w in emoji_options:
          index = emoji_options.index(w)
          emoticons_used.append((w,i+1))
          break
        
      data_extracted = int_to_binary_string(index,bits)
      if verbose:
        print(f'>>>decoding word:"{words[i]}" next word:"{words[i+1]}" length:"{len(emoji_options)}"')
        print(f'bits:"{bits}" data extracted:"{data_extracted}" index:"{index}"')
      bytes_str += data_extracted

    for emo,idx in reversed(emoticons_used):
      s,e = input_str_ss.non_spaces[idx]
      if emo:
        result = result[:s-1] + result[s:e].replace(emo,'') + result[e:]
  
    return result, bytes_str


tests = 100
acc = 0
onlyRatio = False
print(f"Running {tests} tests")
for i in range(tests):
  data = random_bit_stream(60)
  # text = 'hi, how are you?'
  LONG_TEXT = '''Metaphysical solipsism is a variety of solipsism. Based on a philosophy of subjective idealism, metaphysical solipsists maintain that the self is the only existing reality and that all other realities, including the external world and other persons, are representations of that self, and have no independent existence.[citation needed] There are several versions of metaphysical solipsism, such as Caspar Hare's egocentric presentism (or perspectival realism), in which other people are conscious, but their experiences are simply not present.'''
  text = LONG_TEXT
  verbose = False
  encoded_text,rem = Emojier.encode(text,data,verbose=verbose)
  if not onlyRatio:
    print('rem=',rem)
    print('encoded_text=',encoded_text)
  original_txt, deData = Emojier.decode(encoded_text,verbose=verbose)
  if not onlyRatio:
    print('original_txt=',original_txt)
  deData += rem
  if not onlyRatio:
    print(f'text="{text}"\n->\nencoded_text="{encoded_text}" \ndata="{data}"\ndeData="{deData}"\ndata==deData="{data==deData}"')
  ratio =(len(data)-len(rem)) / len(text)
  acc += ratio
  if not onlyRatio:
    print(f'ratio={len(data)-len(rem)} / {len(text)}={ratio}')
  assert data==deData
  assert text==original_txt
  if not onlyRatio:
    print('\n')
    print("#"*100)
    print('\n')

print(f'avg ratio = {acc/tests}')
print(f'old ratio = 0.8235294117648159')

# 0000

Running 100 tests
rem= 000100001000000001011100010111111010010100101
encoded_text= Metaphysical solipsism is a variety of solipsism. Based ✊👌 on a philosophy of subjective idealism, metaphysical solipsists maintain that the self is the only existing reality 💯🔊 and that all other realities, including the external world 🌎 and other persons, are representations of that self, and have no independent existence.[citation needed] There are several versions of metaphysical solipsism, such as Caspar Hare's egocentric presentism (or perspectival realism), in which other people are conscious, but their 👎🏻🏻🍷 experiences are simply not present.
original_txt= Metaphysical solipsism is a variety of solipsism. Based on a philosophy of subjective idealism, metaphysical solipsists maintain that the self is the only existing reality and that all other realities, including the external world and other persons, are representations of that self, and have no independent existence.[citation needed] There are 

In [ ]:
# https://huggingface.co/spaces/adorkin/BilingualEmojiPredictor/blob/main/app.py

%pip install transformers

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import torch

BASE_MODEL = "amazon-sagemaker-community/xlm-roberta-en-ru-emoji-v2"
TOP_N = 5

model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
    
def get_top_emojis(text):
    preprocessed = preprocess(text)
    inputs = tokenizer(preprocessed, return_tensors="pt")
    preds = model(**inputs).logits
    scores = torch.nn.functional.softmax(preds, dim=-1).detach().numpy()
    sorted_scores = [float(value) for value in np.sort(scores.squeeze())[::-1]]
    ranking = np.argsort(scores)
    ranking = ranking.squeeze()[::-1]
    emojis = [model.config.id2label[i] for i in ranking]
    return dict(zip(emojis, sorted_scores))

get_top_emojis(preprocess('I’ve toyed with tghe idea of usuing GPT-3’s API to add much more intelligent capabilirties to RC'))

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import torch

example_prompts = [
    "it's pretty depressing when u hit pan on ur favourite highlighter",
    "After what just happened. In need to smoke.",
    "I've never been happier. I'm laying awake as I watch @user sleep. Thanks for making me happy again, babe.",
    "@user is the man",
    "Поприветствуем моего нового читателя @user",
    "сегодня у одной крутой бичи день рождения! @user поздравляю тебя с днем рождения! будь самой-самой счастливой,красота:* море любви тебе",
    "Никогда не явствовала себя ужаснее, чем сейчас:( я просто раздавленна",
    "Самое ужасное - это ожидание результатов",
    "печально что заряд одинаково фигово держится(",
]



models_to_choose = [
    "amazon-sagemaker-community/xlm-roberta-en-ru-emoji-v2",
    "AlekseyDorkin/xlm-roberta-en-ru-emoji"
]

BASE_MODEL = models_to_choose[0]
TOP_N = 5


def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def load_model():
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL)
    return model, tokenizer

def get_top_emojis(text, top_n=TOP_N):
    model, tokenizer = load_model()
    preprocessed = preprocess(text)
    inputs = tokenizer(preprocessed, return_tensors="pt")
    preds = model(**inputs).logits
    scores = torch.nn.functional.softmax(preds, dim=-1).detach().numpy()
    ranking = np.argsort(scores)
    ranking = ranking.squeeze()[::-1][:top_n]
    emojis = [model.config.id2label[i] for i in ranking]
    return ', '.join(map(str, emojis))
        

model, tokenizer = load_model()

get_top_emojis(preprocess("Hi, how are you?"))
    
    

get_top_emojis(preprocess("I’ve toyed with tghe idea of usuing GPT-3’s API to add much more intelligent capabilirties to RC"))